### KNN Model – Geospatial Gun Violence Analysis

#### Importing Libraries

In [57]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium.plugins import HeatMap
from shapely.geometry import Point
from scipy.spatial import cKDTree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier #https://www.geeksforgeeks.org/machine-learning/k-nearest-neighbours/
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

#### Loading and Preparing the Dataset

In [58]:
data = pd.read_csv("GVA_w-injury_2015_2021apr_geocoded.csv")
data = data.rename(columns={"X": "longitude", "Y": "latitude"})
data['n_killed'] = data['n_killed'].fillna(0)
data['n_injured'] = data['n_injured'].fillna(0)

#### Creating GeoDataFrame and Setting Coordinate Reference System

##### Latitude and longitude values are converted into geometric points and reprojected to a projected coord

In [59]:
data = data.drop(columns=['state', 'city_or_county', 'address', 'incident_url', 'source_url'])

gdf = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data.longitude, data.latitude),
    crs="EPSG:4326"
)
gdf = gdf.to_crs(epsg=3857)

#### Calculating Distance to Nearest Point

##### A KDTree is constructed to allow fast nearest-neighbor and radius-based spatial queries on large datasets.
###### A KD-Tree (k-dimensional tree) is a binary tree data structure used to organize points in k-dimensional space so that nearest-neighbor searches can be performed efficiently.

In [60]:
coords = np.array(list(zip(gdf.geometry.x, gdf.geometry.y)))
tree = cKDTree(coords)


#### Calculating Distance to the Nearest Incident

##### The distance to the closest neighboring incident is calculated using the KDTree. This feature captures local clustering intensity.

In [61]:
distances, indices = tree.query(coords, k=2)
gdf['dist_to_nearest'] = distances[:, 1]

buffer_radius = 2000

#### Calculating Local Neighborhood Statistics (2 km Radius)

##### Neighborhood-level features are computed within a 2 km radius to summarize local incident density and severity.

In [62]:
gdf['count_2km'] = gdf.geometry.apply(
    lambda point: len(tree.query_ball_point([point.x, point.y], r=buffer_radius)) - 1
)

def mean_in_radius(point, radius=buffer_radius):
    neighbors_idx = tree.query_ball_point([point.x, point.y], r=radius)
    neighbors = gdf.iloc[neighbors_idx]
    return pd.Series({
        'mean_killed_2km': neighbors['n_killed'].mean(),
        'mean_injured_2km': neighbors['n_injured'].mean()
    })

gdf[['mean_killed_2km','mean_injured_2km']] = gdf.geometry.apply(lambda pt: mean_in_radius(pt))


#### Defining Features and Splitting Data for Risk Classification

In [63]:
gdf['risk_level'] = ((gdf['n_killed'] > 0) | (gdf['n_injured'] >= 3)).astype(int)
feature_cols = ['latitude', 'longitude', 'month', 'year', 'dist_to_nearest', 'count_2km']
X = gdf[feature_cols]
y = gdf['risk_level']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=53)

#### Training K-Nearest Neighbors Classifier

In [64]:
knn = KNeighborsClassifier(
    n_neighbors=15,
    weights='distance',
    n_jobs=-1
)
knn.fit(X_train, y_train)
gdf["pred_risk"] = knn.predict(gdf[feature_cols])

#### Evaulating accuracy

In [65]:
preds = knn.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, preds))

Validation Accuracy: 0.5870784663103016


##### The accuracy of KNN model on predictions are pretty high with 58.7%

#### Defining a Function to Predict Safety Risk at a Given Location

In [66]:
def predict_safety(lat, lon, month, year):
    point = Point(lon, lat)
    point_proj = gpd.GeoSeries([point], crs="EPSG:4326").to_crs(epsg=3857)
    x, y_ = point_proj.geometry.x.iloc[0], point_proj.geometry.y.iloc[0]

    dist, idx = tree.query([(x, y_)], k=2)
    dist_to_nearest = dist[0][1]

    count_2km = len(tree.query_ball_point((x, y_), r=buffer_radius)) - 1

    X_new = pd.DataFrame([{
        'latitude': lat,
        'longitude': lon,
        'month': month,
        'year': year,
        'dist_to_nearest': dist_to_nearest,
        'count_2km': count_2km
    }])
    pred = knn.predict(X_new)[0]
    return "HIGH RISK" if pred == 1 else "LOW RISK"

#### Visualizing Local Gun Violence Risk Around a Target Location

#### Reprojecting Data for Web Mapping

In [67]:
gdf_folium = gdf.to_crs(epsg=4326)
gdf_viz = gdf_folium.sample(frac=1/20, random_state=42)

#### Initializing the Base Map

In [68]:
m_knn = folium.Map(
    location=[39.5, -98.35],
    zoom_start=4,
    tiles="CartoDB positron"
)

#### Creating a Layer for High-Risk Locations

In [69]:
risk_layer = folium.FeatureGroup(name="High Risk Locations")


#### Plotting High-Risk Incidents as Circle Markers

In [70]:
for _, row in gdf_viz.iterrows():
    if row["pred_risk"] == 1:
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=5,
            color="red",
            fill=True,
            fill_opacity=0.7
        ).add_to(risk_layer)

#### Preparing Data for Heatmap Generation

In [72]:
risk_layer.add_to(m_knn)

heat_data = [
    [row.geometry.y, row.geometry.x, 1]
    for _, row in gdf_viz.iterrows()
    if row["pred_risk"] == 1
]

HeatMap(
    heat_data,
    radius=15,
    blur=20,
    max_zoom=10,
    name="High Risk Density"
).add_to(m_knn)

folium.LayerControl(collapsed=False).add_to(m_knn)

m_knn